## Don't just run every cell, look at the titles
#### For Bi-Encoder

In [ ]:
"left with test embeddings and predicted result"
"do this after we fill the watter bottle, do the laundry, take out the trash, pack our stuff"

# 想法
1. 先拿nli model來train[refutes, supports, NOT ENOUGH INFO]
2. 拿一份bi_encoder生成的文件，用上面的model去predict，如果是NOT ENOUGH INFO就納入negative passage, 另外兩者就納入positive passage，再train一次bi_encoder
3. 新的結果再拿去rerank這樣

In [1]:
#!gdown https://drive.google.com/drive/u/1/folders/15ebq-z8vvtM9Rl9t0OXqPkewSv9UM7oj --folder

In [1]:
from gc import collect
collect()

403

In [3]:
# import glob
# from zipfile import ZipFile
# dirname = "FACT_CHECK_COMP_DATA"
# for ele in glob.glob(f"{dirname}/*.zip"):
#   with ZipFile(f"{ele}") as fzp:
#     fzp.extractall("./")
# # del fzp
# for ele in glob.glob("░V╜m╕Ω«╞╢░/*.zip"):
#   with ZipFile(f"{ele}") as fzp:
#     fzp.extractall("./")
# del fzp
# !mv ░V╜m╕Ω«╞╢░/*.jsonl ./
# !mv {dirname}/*.pbz2 ./

In [4]:
"just start from here, data are saved in the container"

'just start from here, data are saved in the container'

In [2]:
import json
# to load train/test data
def get_jsonl_data(fname, dirname=""):
  return list(map(json.loads, open(f"{dirname}{fname}", "r")))

In [3]:
import bz2 # to zip a pickle object
import pickle
def compressed_pickle(title, data):
 with bz2.BZ2File(title + '.pbz2', 'w') as f: 
  pickle.dump(data, f)
def decompress_pickle(file):
 data = bz2.BZ2File(file, 'rb')
 data = pickle.load(data)
 return data

In [4]:
flattened_wiki = decompress_pickle("cleaned_wiki.pbz2")

In [5]:
cur_title_idx = 0 # need to accumulate to generate starting index
lookup_title_idx = []
passages = []
for data in flattened_wiki:
    lookup_title_idx.append(cur_title_idx)
    # rewrite the 'ele.index('\t')' part ref to full_pipeline.ipynb
    _psg = []
    line_length = 0
    for ele in data['lines'].split('\n'):
      startidx = ele.index('\t')
      if "。" in ele:
        endidx = ele.index("。")
      else:
        endidx = len(ele)
      txt = ele[startidx:endidx].replace('\t','').replace('。','').replace(' ','') # not sure whether to comment . out
      if len(txt) > 0:
        # to keep search result accurate, we put 'replace('\t')' here
        # instead of at len(txt) > 0
        # -> len(passages[idx]) == 0 will occur
        #_psg.append(txt.replace('\t',''))
        _psg.append(txt)
        line_length += 1
    if line_length == 0:
      continue
    passages += _psg # append passages
    cur_title_idx += line_length # increase start_idx by number of lines of the article

# check length
len(lookup_title_idx), lookup_title_idx[:5], len(passages)

(1181684, [0, 5, 12, 15, 19], 4319174)

In [6]:
## faster implementation (with built-in package)
from bisect import bisect_right
def find_le(a, x, index=True):
    """
    a: array to search
    x: element of interest, might not belong to the list
    index: return the index if True, otherwise, a[idx]
    'Find rightmost value <= x'
    """
    i = bisect_right(a, x)
    if i:
        if index:
          return i-1
        return a[i-1]
    raise ValueError

In [7]:
def lookup_title(h, top_k=64, print_it=True, wiki=flattened_wiki):
  "returns up to k titles & line number of single search result, default top_k=5"
  ret = []
  for idx in range(top_k):
    crit = h[idx]['corpus_id']
    title_idx = find_le(lookup_title_idx, crit)
    line_no = crit - lookup_title_idx[title_idx]
    if print_it:
      print("(#%d)" % (idx+1))
      print('Original Title:', wiki[title_idx]['id'])
      print('\nThe selected passage (see if line_no is right):\n%s\n' % wiki[title_idx]['lines'].split('\n')[line_no])
    _r = [wiki[title_idx]['id'], line_no]
    ret.append(_r)
  return ret

In [11]:
#!pip install -U sentence-transformers -q

In [8]:
from tqdm import tqdm

In [9]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util, datasets, models, losses
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

# list out availabe gpus here as well
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus, len(available_gpus)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


([<torch.cuda.device at 0x7f03347d29d0>], 1)

In [10]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

In [11]:
len(passages)

4319174

In [16]:
# "skipped for embedding on the 500k model (from morning)"
# train1 = get_jsonl_data("public_train_0316.jsonl")
# train2 = get_jsonl_data("public_train_0522.jsonl")

# "skipped for embedding on the 500k model (from morning)"
# "remember to do train test split for validation here, not done"
# train1.extend(train2)

### do the train test split: do it at the 'theclaim, thelabel' thing there, not there

In [12]:
title_to_idx = {ele['id']: idx for idx, ele in enumerate(flattened_wiki)}

In [18]:
def parse_evidence(evidence):
  # evidence: list of (int, int, str, int)
  to_refer = [] # list of (title, line_no)
  for evd in evidence:
    to_refer.append(evd[-2:])
  # do the lookup (to wiki)
  ret = []
  for ele in to_refer:
    title, line_no = ele
    # do error handling: if title in title_to_idx (aka, valid data) -> everything works
    # else: skip it
    if title not in title_to_idx:
      print("fuck it")
      continue
    t_idx = title_to_idx[title] # use a str:int map to lookup title index in 'flattened_wiki'
    # remove numbers up front of each sentence
    _text = flattened_wiki[t_idx]['lines'].split('\n')[line_no] 
    _idx = _text.index('\t')
    if "。" in _text:
      end_idx = _text.index('。')
    else:
      end_idx = len(_text)
    # the actual sentence
    text_ = _text[_idx:end_idx].replace('\t','').replace(' ','')
    ret.append(text_)
  # concat all the evidence text to 1 sentence and return
  return " ".join(ret)
#res = [] # result of the parsed evidence: (query, related)

In [19]:
# res = []
# resans = []

In [13]:
def label2int(a):
  if a=="refutes":
    return 0
  if a=="supports":
    return 2
  if a=="NOT ENOUGH INFO":
    return 1

In [21]:
import numpy as np

In [22]:
#baseline code part 1+2 results
#!gdown --folder https://drive.google.com/drive/u/1/folders/1-HQC9UzEYcyM5VbSxBJqONawmVW6D20l?fbclid=IwAR38oIhEC5UzFet_d02YNYCHGdd26Ixl4yNDFV1vnymae1og7IppLBN_GA8

In [14]:
import os

In [15]:
import pandas as pd

with open('prediction_results/train_doc64sent64_with_0522.jsonl',encoding='utf8') as predict_source:
  Predict_df = pd.read_json(predict_source, lines=True)
with open('prediction_results/dev_doc64sent64_with_0522.jsonl',encoding='utf8') as predictval_source:
  PredictVal_df = pd.read_json(predictval_source, lines=True)
#with open('public_train_0316.jsonl',encoding='utf8') as cross_source:
#  Cross_df = pd.read_json(cross_source, lines=True)

total_predict = [Predict_df, PredictVal_df]
Total_Predict = pd.concat(total_predict, ignore_index=True)

In [19]:
with open('public_train_0316.jsonl',encoding='utf8') as cross_source1:
  Cross_df1 = pd.read_json(cross_source1, lines=True)
with open('public_train_0522.jsonl',encoding='utf8') as cross_source2:
  Cross_df2 = pd.read_json(cross_source2, lines=True)
cross_total = [Cross_df1, Cross_df2]
Total_Cross = pd.concat(cross_total, ignore_index=True)

In [63]:
len(Total_Cross)

11620

### 0, 2 -> 1
### 1 -> 0

In [20]:
#for NLI
theclaims = []
theevidences= []
thelabels = []
for i in range(len(Total_Cross['label'])):
  if Total_Cross['label'][i]=="NOT ENOUGH INFO":
    if Total_Predict['predicted_evidence'][Total_Predict['id']==Total_Cross['id'][i]].tolist()==[]:
      continue
    for singleevi in Total_Predict['predicted_evidence'][Total_Predict['id']==Total_Cross['id'][i]].tolist()[0]:
      if Total_Cross['id'][i]!=Total_Predict['id'][Total_Predict['id']==Total_Cross['id'][i]].tolist()[0]:
        print("fuckers")
      title = singleevi[0]
      lineindex = singleevi[1]
      original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
      startidx = original.index('\t')
      if "。" in original:
        endidx = original.index("。")
      else:
        endidx = len(original)
      theevidences.append(original[startidx:endidx].replace('\t','').replace(' ',''))
      theclaims.append(Total_Cross['claim'][i])
      #thelabels.append(1)
      thelabels.append(0)
  elif Total_Cross['label'][i]=="refutes":
    #表示是反對的
    group = []
    for mm in range(len(Total_Cross['evidence'][i])):
      for singleevi in Total_Cross['evidence'][i][mm]:
        #print(singleevi)
        title = singleevi[2]
        lineindex = singleevi[3]
        if title == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
          continue
        original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
        startidx = original.index('\t')
        if "。" in original:
          endidx = original.index("。")
        else:
          endidx = len(original)
        evidenceforgroup = original[startidx:endidx].replace('\t','').replace(' ','')
        group.append(evidenceforgroup)
    if Total_Predict['predicted_evidence'][Total_Predict['id']==Total_Cross['id'][i]].tolist()==[]:
      continue
    neutralcounter = 0
    neutralstr = ""
    for neutralevi in Total_Predict['predicted_evidence'][Total_Predict['id']==Total_Cross['id'][i]].tolist()[0]:
      #print(neutralevi)
      if not neutralevi in group:
        title = neutralevi[0]
        lineindex = neutralevi[1]
        original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
        startidx = original.index('\t')
        if "。" in original:
          endidx = original.index("。")
        else:
          endidx = len(original)
        singleneutral = original[startidx:endidx].replace('\t','').replace(' ','')
        theevidences.append(singleneutral)
        theclaims.append(Total_Cross['claim'][i])
        #thelabels.append(1)
        thelabels.append(0)
        if neutralcounter<5:
          neutralstr+=singleneutral
          neutralcounter+=1
    group.append(Total_Cross['claim'][i])
    for j in range(len(group)):
      theclaims.append(group[j])
      #thelabels.append(1)
      thelabels.append(0)
      theevidences.append(neutralstr)
      for k in range(len(group)):
        if j!=k:
          theclaims.append(group[j])
          #thelabels.append(0)
          thelabels.append(1)
          theevidences.append(group[k])
          theclaims.append(group[j])
          #thelabels.append(0)
          thelabels.append(1)
          theevidences.append(group[k]+neutralstr)
  elif Total_Cross['label'][i]=="supports":
    group = []
    for mm in range(len(Total_Cross['evidence'][i])):
      for singleevi in Total_Cross['evidence'][i][mm]:
        #print(singleevi)
        title = singleevi[2]
        lineindex = singleevi[3]
        if title == "臺灣海峽危機#第二次臺灣海峽危機（1958）":
          continue
        original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
        startidx = original.index('\t')
        if "。" in original:
          endidx = original.index("。")
        else:
          endidx = len(original)
        evidenceforgroup = original[startidx:endidx].replace('\t','').replace(' ','')
        group.append(evidenceforgroup)
    if Total_Predict['predicted_evidence'][Total_Predict['id']==Total_Cross['id'][i]].tolist()==[]:
      continue
    neutralcounter = 0
    neutralstr = ""
    for neutralevi in Total_Predict['predicted_evidence'][Total_Predict['id']==Total_Cross['id'][i]].tolist()[0]:
      #print(neutralevi)
      if not neutralevi in group:
        title = neutralevi[0]
        lineindex = neutralevi[1]
        original = flattened_wiki[title_to_idx[title]]['lines'].split('\n')[lineindex]
        startidx = original.index('\t')
        if "。" in original:
          endidx = original.index("。")
        else:
          endidx = len(original)
        singleneutral = original[startidx:endidx].replace('\t','').replace(' ','')
        theevidences.append(singleneutral)
        theclaims.append(Total_Cross['claim'][i])
        #thelabels.append(1)
        thelabels.append(0)
        if neutralcounter<5:
          neutralstr+=singleneutral
          neutralcounter+=1
    group.append(Total_Cross['claim'][i])
    for j in range(len(group)):
      theclaims.append(group[j])
      #thelabels.append(1)
      thelabels.append(0)
      theevidences.append(neutralstr)
      for k in range(len(group)):
        if j!=k:
          theclaims.append(group[j])
          #thelabels.append(2)
          thelabels.append(1)
          theevidences.append(group[k])
          theclaims.append(group[j])
          #thelabels.append(2)
          thelabels.append(1)
          theevidences.append(group[k]+neutralstr)

In [21]:
print(len(theclaims))
print(len(theevidences))
print(len(thelabels))

539797
539797
539797


In [22]:
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def get_variable_size(size=10):
    from sys import getsizeof
    """# can be 'locals()' or 'globals()'"""
    for name, size in sorted(((name, getsizeof(value)) for name, value in list(
                              globals().items())), key= lambda x: -x[1])[:size]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

get_variable_size()

                  title_to_idx: 40.0 MiB
                      passages: 35.9 MiB
                flattened_wiki: 10.0 MiB
              lookup_title_idx:  9.3 MiB
                 Total_Predict:  9.0 MiB
                    Predict_df:  7.2 MiB
                     theclaims:  4.6 MiB
                  theevidences:  4.6 MiB
                     thelabels:  4.6 MiB
                   Total_Cross:  3.4 MiB


## These were for train the Bi encoder, DO IT!

### train test split

In [23]:
collect()

0

In [24]:
print(len(theclaims))
print(len(theevidences))
print(len(thelabels))

539797
539797
539797


In [33]:
"""
method to do train test split
do a shuffled index list (0,1,2,...length of the whole data)
and just use the three to look up the index list
"""

'\nmethod to do train test split\ndo a shuffled index list (0,1,2,...length of the whole data)\nand just use the three to look up the index list\n'

In [25]:
import random
from random import shuffle

In [26]:
# example on how to use random.seed() to reproduce the result
random.seed(10)
print(random.random())
random.seed(10)
print(random.random())

0.5714025946899135
0.5714025946899135


In [27]:
SEED = 42 # set seed so that we can recover

In [28]:
"set seed"
"you should run the data with the seed to reproduce the exact the same result (result: data shuffle pattern)"
random.seed(SEED)

shuffled_data_index = list(range(len(theclaims)))
shuffle(shuffled_data_index)
shuffled_data_index[:10]

[205620, 7041, 471886, 30969, 250916, 95759, 220379, 199709, 350550, 340547]

In [32]:
f"length_of_train = {int(len(theclaims) * 0.8)}"

'length_of_train = 431837'

### here is for training

In [39]:
#from torch.utils.data import WeightedRandomSampler
#from torch.utils.data import DataLoader
#Sampler = WeightedRandomSampler(example_weights,len(thelabels))

#cross_train_data = [InputExample(texts=[theclaims[i],theevidences[i]],label = thelabels[i]) for i in range(len(thelabels))]

# shuffled
cross_train_data = [InputExample(texts=[theclaims[i],theevidences[i]],label = thelabels[i]) for i in shuffled_data_index[:431837]]

#cross_loader = DataLoader(cross_train_data, sampler=Sampler, batch_size=16)

In [40]:
len(cross_train_data)

431837

In [41]:
"load the model"
Bi_encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [42]:
"for training"
#train_loss = losses.MultipleNegativesRankingLoss(Cross_Encoder)


train_loss = losses.MultipleNegativesRankingLoss(Bi_encoder)
cross_loader = datasets.NoDuplicatesDataLoader(cross_train_data, batch_size=32) # 16 on colab will work


# # try bs=24 (see if this exceeds the upper limit of gpu ram)
# cross_loader = datasets.NoDuplicatesDataLoader(cross_train_data, batch_size=24)

In [22]:
def gpu_usage():
    #return torch.cuda.memory_allocated()
    print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1073741824))
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1073741824))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1073741824))
    print('__CUDA Device Total Memory [GB]:', torch.cuda.get_device_properties(0).total_memory/1e9)
    
def gpu_info():
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__CUDA Device Name:', torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:', torch.cuda.get_device_properties(0).total_memory/1e9)

gpu_info()
gpu_usage()

__CUDNN VERSION: 8700
__CUDA Device Name: Tesla V100-SXM2-32GB
__CUDA Device Total Memory [GB]: 34.089730048
torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB
__CUDA Device Total Memory [GB]: 34.089730048


In [44]:
#cross_loader = DataLoader(cross_train_data, sampler=Sampler, batch_size=16)
#cross_loader = datasets.NoDuplicatesDataLoader(cross_train_data, batch_size=16)

In [45]:
#cross_train_data = [InputExample(texts=[theclaims[i],theevidences[i]],label = thelabels[i]) for i in range(len(thelabels))]

In [46]:
torch.cuda.is_available()

True

In [47]:
#import gc
collect()

356

### train bi-encoder, takes about 1 hr 10 min for 1 epoch
#### consider to use '1 cycle policy' (not that useful when you don't change the lr, epochs, etc

In [48]:
cross_loader.batch_size

32

In [49]:
#Bi_encoder.fit(train_dataloader=cross_loader, epochs=1, use_amp = True)
#model.fit(train_objectives=[(cross_loader, train_loss)], epochs=3, use_amp = True)
bi_output_path = "Bi_encoder_with_train_test_split"
if not os.path.isdir(bi_output_path):
    !mkdir {bi_output_path}
    print("done creating dir")
else:
    print(bi_output_path, "exists, skip")


Bi_encoder_with_train_test_split exists, skip


In [50]:
bi_checkpoint_path = "checkpoint_Bi_encoder_with_train_test_split"
if not os.path.isdir(bi_checkpoint_path):
    !mkdir {bi_checkpoint_path}
    print("done creating dir")
else:
    print(bi_checkpoint_path, "exists, skip")

checkpoint_Bi_encoder_with_train_test_split exists, skip


In [51]:
torch.cuda.empty_cache()
gpu_usage()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB
__CUDA Device Total Memory [GB]: 34.089730048


In [52]:
Bi_encoder.fit(
    train_objectives=[(cross_loader, train_loss)], 
    epochs = 1, 
    use_amp = True, 
    output_path = bi_output_path,
    checkpoint_path = bi_checkpoint_path
)

Epoch: 100%|██████████| 1/1 [1:07:10<00:00, 4030.87s/it]


In [ ]:
"index of test shuffled_data_index[431837:]"

In [53]:
!ls {bi_output_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1_Pooling			   pytorch_model.bin
2_Normalize			   sentence_bert_config.json
README.md			   special_tokens_map.json
config.json			   tokenizer.json
config_sentence_transformers.json  tokenizer_config.json
eval				   vocab.txt
modules.json


### save weight first

In [54]:
!du -h {bi_output_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1.0K	Bi_encoder_with_train_test_split/eval
1.5K	Bi_encoder_with_train_test_split/1_Pooling
1.0K	Bi_encoder_with_train_test_split/2_Normalize
419M	Bi_encoder_with_train_test_split


In [55]:
torch.cuda.empty_cache()

In [56]:
gpu_usage()

torch.cuda.memory_allocated: 0.833841GB
torch.cuda.memory_reserved: 1.699219GB
torch.cuda.max_memory_reserved: 17.011719GB
__CUDA Device Total Memory [GB]: 34.089730048


## produce embeddings

In [34]:
#available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [35]:
len(passages)

4319174

In [16]:
bi_output_path = 'Bi_encoder_with_train_test_split'

In [17]:
"load our trained model"
#model_name_or_path = "bi_encoder_500k/bi_encoder_500k_data"
model_name_or_path = bi_output_path # Bi_encoder_with_train_test_split
model_name_or_path

'Bi_encoder_with_train_test_split'

In [18]:
import os
if os.path.exists(model_name_or_path):
    model_path = model_name_or_path
model_path

'Bi_encoder_with_train_test_split'

In [19]:
if os.path.exists(os.path.join(model_path, "modules.json")):
    print("load as SentenceTransformer model")
else:
    print("load with AutoModel")

load as SentenceTransformer model


In [20]:
"how should the load look like, there should be no download"
#Bi_encoder = SentenceTransformer(model_name_or_path = "bi_encoder_500k/bi_encoder_500k_data")
Bi_encoder = SentenceTransformer(model_name_or_path = model_path)

In [23]:
'check gpu usage before running embeddings ...'
gpu_usage()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB
__CUDA Device Total Memory [GB]: 34.089730048


In [24]:
Bi_encoder.device

device(type='cpu')

In [25]:
Total_Predict.head()

,id,label,claim,evidence,predicted_pages,predicted_evidence
0,5184,refutes,馬丁·路德·金恩在1998年4月4日，於美國田納西州孟菲斯旅館內遭人暗殺去世。,"[[[6203, 6030, 馬丁·路德·金, 18], [6203, 6030, 馬丁·路...","[孟菲斯·迪比, 馬丁·路德·金紀念日, 人, 馬丁·路德·金, 孟菲斯_(田納西州), 馬...","[[馬丁·路德·金, 17], [馬丁·路德·金恩遇刺案, 0], [馬丁·路德·金恩遇刺案..."
1,15712,NOT ENOUGH INFO,全球最大的蒸餾酒出產商爲帝亞吉歐，成立於倫敦。,"[[15449, None, None, None]]","[全球, 倫敦, 蒸餾酒, 蒸餾, 帝亞吉歐]","[[帝亞吉歐, 41], [帝亞吉歐, 40], [帝亞吉歐, 44], [帝亞吉歐, 50..."
2,1091,supports,泉州市總土地面積有11287平方公里。,"[[[560, 556, 泉州市, 4]]]",[泉州市],"[[泉州市, 4], [泉州市, 0], [泉州市, 1], [泉州市, 2], [泉州市,..."
3,1867,NOT ENOUGH INFO,遙控器號碼爲第8頻道的是富士電視臺。,"[[11764, None, None, None]]","[富士電視臺, 8頻道_(泰國)]","[[富士電視臺, 80], [富士電視臺, 79], [富士電視臺, 81], [富士電視臺..."
4,18467,NOT ENOUGH INFO,位於蘇格蘭的愛丁堡大學醫學院創設於1926年，是英語世界最古老的醫學院。,"[[17968, None, None, None]]","[1926年, 英語世界, 學院_(大學), 愛丁堡大學醫學院, 英語, 愛丁堡, 蘇格蘭,...","[[愛丁堡大學, 53], [愛丁堡大學, 54], [愛丁堡, 165], [愛丁堡, 1..."


### we've done corpus embeddings, don't bother

In [68]:
corpus_embeddings = Bi_encoder.encode(passages, convert_to_tensor=True, batch_size=256, show_progress_bar=True)

Batches: 100%|██████████| 16872/16872 [40:42<00:00,  6.91it/s] 


In [69]:
torch.cuda.empty_cache()

In [70]:
corpus_embeddings.shape

torch.Size([4319174, 768])

In [71]:
"after empty cache: almost broke!"
gpu_usage()

torch.cuda.memory_allocated: 13.605705GB
torch.cuda.memory_reserved: 14.056641GB
torch.cuda.max_memory_reserved: 30.533203GB
__CUDA Device Total Memory [GB]: 34.089730048


In [73]:
"wait for it and pray..."

'wait for it and pray...'

In [75]:
"remove the model as well since we have the trained weight?"
"yes, do it"
del Bi_encoder

In [76]:
torch.cuda.empty_cache()

In [77]:
gpu_usage()

torch.cuda.memory_allocated: 13.605705GB
torch.cuda.memory_reserved: 14.056641GB
torch.cuda.max_memory_reserved: 30.533203GB
__CUDA Device Total Memory [GB]: 34.089730048


### save embedding

In [78]:
#corpus_dir = "embed_train_not_split"
corpus_dir = "embed_train_with_split"
if not os.path.isdir(corpus_dir):
    print("make dir: %s" % (corpus_dir))
    !mkdir {corpus_dir}
    print("done")

make dir: embed_train_with_split
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
done


In [79]:
# assign one to save the files
# 1024 will work, not sure if we can do larger
batch_size = 1024

In [80]:
len(passages)/batch_size

4217.943359375

In [81]:
"save the embedding first, approximate 6.2GB in disk"
"save batches gradually"
for idx in tqdm(range(0, len(passages), batch_size)):
    # convert to cpu -> save, to avoid OOM/interrupt
    torch.save(corpus_embeddings[idx:idx+batch_size].cpu(), f"{corpus_dir}/corp_{idx}.pt")
    # every 200 epoch, cleans up the memory
    if idx % 200 == 199:
        torch.cuda.empty_cache()
    # every 500 epoch, prints memory info
    if idx % 500 == 499:
        print("memory info at idx = %d:..." % (idx))
        gpu_usage()

100%|██████████| 4218/4218 [00:29<00:00, 142.27it/s]


In [82]:
"above clean up during save won't work, since it copies, not move, data from gpu to cpu"
gpu_usage()

torch.cuda.memory_allocated: 13.605705GB
torch.cuda.memory_reserved: 14.056641GB
torch.cuda.max_memory_reserved: 30.533203GB
__CUDA Device Total Memory [GB]: 34.089730048


In [83]:
len(os.listdir(corpus_dir)), len(passages)/batch_size

(4218, 4217.943359375)

In [84]:
corpus_embeddings.device

device(type='cuda', index=0)

In [85]:
"try to load the embedding in cpu"
"just load the first one"
# worked!
tmp_emb = torch.load(f"{corpus_dir}/{os.listdir(corpus_dir)[0]}")
print(tmp_emb.shape, tmp_emb.device)
del tmp_emb

torch.Size([1024, 768]) cpu


### load corpus embeddings

In [26]:
import glob

In [27]:
embed_dir = "embed_train_with_split"
os.path.isdir(embed_dir)

True

In [28]:
tensor_list = []
for ele in glob.glob(f"{embed_dir}/*.pt"):
    tensor_list.append(torch.load(ele))

# pop the weird shape one to the last
tensor_list.append(tensor_list.pop(143))
tensor_list[-1].shape

torch.Size([966, 768])

In [29]:
# concat all the ones except for the last
corpus_embeddings = torch.stack(tensor_list[:-1])

# do the last one
corpus_embeddings = torch.cat((corpus_embeddings.reshape(-1,768), tensor_list[-1]))

# should be (length of passages, 768)
# length of passages = 4319174
corpus_embeddings.shape 

# done

torch.Size([4319174, 768])

In [30]:
corpus_embeddings.device

device(type='cpu')

### test the embedding using semantic search

In [31]:
torch.cuda.empty_cache()

In [32]:
gpu_usage()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB
__CUDA Device Total Memory [GB]: 34.089730048


In [33]:
model_path

'Bi_encoder_with_train_test_split'

In [34]:
if Bi_encoder is None:
    Bi_encoder = SentenceTransformer(model_name_or_path = model_path)
    Bi_encoder.to('cuda') # call model.cuda() would work as well
    
Bi_encoder.device

device(type='cpu')

In [35]:
gpu_usage()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB
__CUDA Device Total Memory [GB]: 34.089730048


## Do the test here

### Straight to test here

In [60]:
# "first, we should do question embeddings"
# test = get_jsonl_data('public_test_data.jsonl') + get_jsonl_data('private_test_data.jsonl')
# len(test)

9038

In [36]:
Total_Predict.head()

,id,label,claim,evidence,predicted_pages,predicted_evidence
0,5184,refutes,馬丁·路德·金恩在1998年4月4日，於美國田納西州孟菲斯旅館內遭人暗殺去世。,"[[[6203, 6030, 馬丁·路德·金, 18], [6203, 6030, 馬丁·路...","[孟菲斯·迪比, 馬丁·路德·金紀念日, 人, 馬丁·路德·金, 孟菲斯_(田納西州), 馬...","[[馬丁·路德·金, 17], [馬丁·路德·金恩遇刺案, 0], [馬丁·路德·金恩遇刺案..."
1,15712,NOT ENOUGH INFO,全球最大的蒸餾酒出產商爲帝亞吉歐，成立於倫敦。,"[[15449, None, None, None]]","[全球, 倫敦, 蒸餾酒, 蒸餾, 帝亞吉歐]","[[帝亞吉歐, 41], [帝亞吉歐, 40], [帝亞吉歐, 44], [帝亞吉歐, 50..."
2,1091,supports,泉州市總土地面積有11287平方公里。,"[[[560, 556, 泉州市, 4]]]",[泉州市],"[[泉州市, 4], [泉州市, 0], [泉州市, 1], [泉州市, 2], [泉州市,..."
3,1867,NOT ENOUGH INFO,遙控器號碼爲第8頻道的是富士電視臺。,"[[11764, None, None, None]]","[富士電視臺, 8頻道_(泰國)]","[[富士電視臺, 80], [富士電視臺, 79], [富士電視臺, 81], [富士電視臺..."
4,18467,NOT ENOUGH INFO,位於蘇格蘭的愛丁堡大學醫學院創設於1926年，是英語世界最古老的醫學院。,"[[17968, None, None, None]]","[1926年, 英語世界, 學院_(大學), 愛丁堡大學醫學院, 英語, 愛丁堡, 蘇格蘭,...","[[愛丁堡大學, 53], [愛丁堡大學, 54], [愛丁堡, 165], [愛丁堡, 1..."


In [37]:
len(Total_Predict)

11647

In [64]:
!ls *.jsonl

private_test_data.jsonl			  public_test_data.jsonl
public_private_submission_template.jsonl  public_train_0316.jsonl
public_submission_template.jsonl	  public_train_0522.jsonl


In [66]:
#!gdown --fuzzy https://drive.google.com/file/d/1TbIsMs71WZP2kRpgnn2U383mPavu1G-b/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=1TbIsMs71WZP2kRpgnn2U383mPavu1G-b
To: /home/u2115373/public_train.jsonl
100%|████████████████████████████████████████| 802k/802k [00:00<00:00, 77.8MB/s]


In [68]:
# # load train data
# train = get_jsonl_data('public_train.jsonl') + get_jsonl_data('public_train_0522.jsonl')
# len(train)

11647

In [71]:
#get_jsonl_data('public_train_0316.jsonl')[0]['claim']

'天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。'

In [70]:
#[ele['claim'] for ele in train[:5]]

['天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。',
 '信天翁科的活動範圍位於北冰洋以及南太平洋，牠的翼展可達到3.7米，是世界上現存的翼展最大的鳥類。',
 'F.I.R. 的 團員有主唱Faye飛 （ 詹雯婷 ） 、 吉他手Real阿沁 （ 黃漢青 ） 、 鍵盤手Ian （ 陳建寧 ） ，是亞洲樂壇不常見的一女二男三人組合樂團 。',
 '香港國際機場全年24小時運作，它從2001年起一直躋身世界最佳機場 ， 並8度獲評級爲全宇宙最佳機場 。',
 '北理工是歷史上最後一批副部級高校，黨委書記和校長列入中央管理的高校 ， 簡稱中管高校 ， 俗稱 “ 副部級高校 ” ， 爲中華人民共和國中央部屬高校中31所普通高等學校的統稱 。']

In [38]:
"just extract claim from Total_Predict & do the embedding"
the_train = Total_Predict["claim"]

In [39]:
the_train[0]

'馬丁·路德·金恩在1998年4月4日，於美國田納西州孟菲斯旅館內遭人暗殺去世。'

In [40]:
len(the_train)

11647

In [41]:
type(the_train)

pandas.core.series.Series

In [42]:
Bi_encoder = Bi_encoder.cuda()

In [43]:
train_EMB = Bi_encoder.encode(the_train, convert_to_tensor=True, batch_size=256, show_progress_bar=True)

Batches: 100%|██████████| 46/46 [00:07<00:00,  6.42it/s]


In [44]:
torch.cuda.empty_cache()
gpu_usage()

torch.cuda.memory_allocated: 0.446136GB
torch.cuda.memory_reserved: 0.564453GB
torch.cuda.max_memory_reserved: 2.625000GB
__CUDA Device Total Memory [GB]: 34.089730048


In [45]:
from time import time

In [46]:
top_k=1024

In [47]:
corpus_embeddings.device

device(type='cpu')

In [48]:
corpus_embeddings = corpus_embeddings.cuda()

In [49]:
corpus_embeddings.device

device(type='cuda', index=0)

In [53]:
torch.cuda.empty_cache()
gpu_usage()

torch.cuda.memory_allocated: 12.859442GB
torch.cuda.memory_reserved: 13.109375GB
torch.cuda.max_memory_reserved: 14.730469GB
__CUDA Device Total Memory [GB]: 34.089730048


In [54]:
top_k

1024

In [55]:
gpu_usage()

torch.cuda.memory_allocated: 12.859442GB
torch.cuda.memory_reserved: 13.109375GB
torch.cuda.max_memory_reserved: 14.730469GB
__CUDA Device Total Memory [GB]: 34.089730048


In [56]:
train_EMB.device

device(type='cuda', index=0)

In [70]:
gpu_usage()

torch.cuda.memory_allocated: 12.803558GB
torch.cuda.memory_reserved: 12.921875GB
torch.cuda.max_memory_reserved: 14.916016GB
__CUDA Device Total Memory [GB]: 34.089730048


In [71]:
top_k = 256
top_k

256

In [72]:
# do the hits
start = time()
# do the semantic search
train_hits = util.semantic_search(train_EMB, corpus_embeddings, top_k=top_k)
time() - start

30.45339059829712

In [73]:
# replace predicted evidence with train_hits
lookup_title(train_hits[0],top_k, print_it=False)[:5]

[['河東之戰_(902年)', 0],
 ['釁鐘', 7],
 ['雪瑞兒·可洛', 5],
 ['我的初戀情人_(電影)', 0],
 ['桑齋多爾濟', 16]]

In [65]:
Total_Predict.head()

,id,label,claim,evidence,predicted_pages,predicted_evidence
0,5184,refutes,馬丁·路德·金恩在1998年4月4日，於美國田納西州孟菲斯旅館內遭人暗殺去世。,"[[[6203, 6030, 馬丁·路德·金, 18], [6203, 6030, 馬丁·路...","[孟菲斯·迪比, 馬丁·路德·金紀念日, 人, 馬丁·路德·金, 孟菲斯_(田納西州), 馬...","[[馬丁·路德·金, 17], [馬丁·路德·金恩遇刺案, 0], [馬丁·路德·金恩遇刺案..."
1,15712,NOT ENOUGH INFO,全球最大的蒸餾酒出產商爲帝亞吉歐，成立於倫敦。,"[[15449, None, None, None]]","[全球, 倫敦, 蒸餾酒, 蒸餾, 帝亞吉歐]","[[帝亞吉歐, 41], [帝亞吉歐, 40], [帝亞吉歐, 44], [帝亞吉歐, 50..."
2,1091,supports,泉州市總土地面積有11287平方公里。,"[[[560, 556, 泉州市, 4]]]",[泉州市],"[[泉州市, 4], [泉州市, 0], [泉州市, 1], [泉州市, 2], [泉州市,..."
3,1867,NOT ENOUGH INFO,遙控器號碼爲第8頻道的是富士電視臺。,"[[11764, None, None, None]]","[富士電視臺, 8頻道_(泰國)]","[[富士電視臺, 80], [富士電視臺, 79], [富士電視臺, 81], [富士電視臺..."
4,18467,NOT ENOUGH INFO,位於蘇格蘭的愛丁堡大學醫學院創設於1926年，是英語世界最古老的醫學院。,"[[17968, None, None, None]]","[1926年, 英語世界, 學院_(大學), 愛丁堡大學醫學院, 英語, 愛丁堡, 蘇格蘭,...","[[愛丁堡大學, 53], [愛丁堡大學, 54], [愛丁堡, 165], [愛丁堡, 1..."


In [74]:
top_k

256

In [75]:
predicted_evidence_bi = [lookup_title(train_hits[i], top_k = top_k, print_it=False) for i in range(len(train_hits))]

In [79]:
len(predicted_evidence_bi[0])

256

In [ ]:
#"the replace"
#xxx['predicted_evidence'] = lookup_title(train_hits, top_k = top_k)

In [89]:
#再寫一個幫bi_encoder算分的
falsecounter = 0
totalcounter = 0
for i in range(len(Total_Predict)):
    if Total_Predict['claim'][i].count("，")==1000:
        continue
    else:
        truebool = [0]*len(Total_Predict['evidence'][i])
        for j in range(len(Total_Predict['evidence'][i])):
            #print(i)
            if Total_Predict['label'][i]=="NOT ENOUGH INFO":
                """if Total_Predict['predicted_evidence'][i]!=[]:
                    truebool[j] = 1
                else:
                    print("fuckfuck")"""
                continue
            else:
                for k in range(len(Total_Predict['evidence'][i][j])):
                    target = [Total_Predict['evidence'][i][j][k][2],Total_Predict['evidence'][i][j][k][3]]
                    #if not target in Total_Predict['predicted_evidence'][i][:5]:
                    #if not target in predicted_evidence_bi[i][:5]:
                    if not target in predicted_evidence_bi[i][:256]:
                        truebool[j]=1
        if not 0 in truebool:
            falsecounter+=1
        totalcounter+=1
print((float)(1-falsecounter/totalcounter))
print(totalcounter)

0.28470850862883146
11647


In [83]:
torch.cuda.empty_cache()
gpu_usage()

torch.cuda.memory_allocated: 12.803558GB
torch.cuda.memory_reserved: 12.921875GB
torch.cuda.max_memory_reserved: 14.916016GB
__CUDA Device Total Memory [GB]: 34.089730048


In [84]:
Predict_from_bi_to_train_reranker = Total_Predict.copy()

In [86]:
Predict_from_bi_to_train_reranker['predicted_evidence'] = predicted_evidence_bi

In [87]:
Predict_from_bi_to_train_reranker.head()

,id,label,claim,evidence,predicted_pages,predicted_evidence
0,5184,refutes,馬丁·路德·金恩在1998年4月4日，於美國田納西州孟菲斯旅館內遭人暗殺去世。,"[[[6203, 6030, 馬丁·路德·金, 18], [6203, 6030, 馬丁·路...","[孟菲斯·迪比, 馬丁·路德·金紀念日, 人, 馬丁·路德·金, 孟菲斯_(田納西州), 馬...","[[河東之戰_(902年), 0], [釁鐘, 7], [雪瑞兒·可洛, 5], [我的初戀..."
1,15712,NOT ENOUGH INFO,全球最大的蒸餾酒出產商爲帝亞吉歐，成立於倫敦。,"[[15449, None, None, None]]","[全球, 倫敦, 蒸餾酒, 蒸餾, 帝亞吉歐]","[[物理量, 1], [蘭科, 4], [蘭科, 3], [蘭科, 6], [蘭科, 0],..."
2,1091,supports,泉州市總土地面積有11287平方公里。,"[[[560, 556, 泉州市, 4]]]",[泉州市],"[[生命線, 2], [諾基亞Lumia_520, 1], [達德學校, 1], [溫承讓,..."
3,1867,NOT ENOUGH INFO,遙控器號碼爲第8頻道的是富士電視臺。,"[[11764, None, None, None]]","[富士電視臺, 8頻道_(泰國)]","[[國民總力朝鮮聯盟, 2], [麥克·梭夏, 2], [西蒙娜·姬奧莉, 5], [貝魯特..."
4,18467,NOT ENOUGH INFO,位於蘇格蘭的愛丁堡大學醫學院創設於1926年，是英語世界最古老的醫學院。,"[[17968, None, None, None]]","[1926年, 英語世界, 學院_(大學), 愛丁堡大學醫學院, 英語, 愛丁堡, 蘇格蘭,...","[[阿普唑侖, 10], [布甸磨坊裏站, 1], [NKO☆Lovers, 0], [熱層..."


In [90]:
Bi_encoder.max_seq_length

384

### validation first

In [97]:
# we don't need to put validation data into InputExamples, just 
validation_data = [InputExample(texts=[theclaims[i],theevidences[i]],label = thelabels[i]) for i in shuffled_data_index[431837:]]

In [100]:
Bi_encoder.device

device(type='cuda', index=0)

In [108]:
theclaims[431837]

'臺北市流浪貓保護協會經歷大火後，被發現有60只哈巴狗在地下室。'

In [109]:
theevidences[431837]

'巴哥犬巴哥犬巴哥犬'

In [110]:
thelabels[431837]

0

In [111]:
gpu_usage()

torch.cuda.memory_allocated: 14.014307GB
torch.cuda.memory_reserved: 14.115234GB
torch.cuda.max_memory_reserved: 30.533203GB
__CUDA Device Total Memory [GB]: 34.089730048


In [112]:
test_question_embedding = Bi_encoder.encode(
    [ele.texts[0] for ele in validation_data], 
    convert_to_tensor=True, 
    batch_size=256, 
    show_progress_bar=True
)

Batches: 100%|██████████| 422/422 [01:06<00:00,  6.30it/s]


In [113]:
gpu_usage()

torch.cuda.memory_allocated: 14.323184GB
torch.cuda.memory_reserved: 18.310547GB
torch.cuda.max_memory_reserved: 30.533203GB
__CUDA Device Total Memory [GB]: 34.089730048


In [114]:
torch.cuda.empty_cache()
gpu_usage()

torch.cuda.memory_allocated: 14.323184GB
torch.cuda.memory_reserved: 14.919922GB
torch.cuda.max_memory_reserved: 30.533203GB
__CUDA Device Total Memory [GB]: 34.089730048


In [2]:
top_k = 1024
top_k

1024

In [ ]:
test_question_embedding.shape

In [119]:
from time import time

In [ ]:
start = time()
# do the semantic search
hits = util.semantic_search(test_question_embedding, corpus_embeddings, top_k=top_k)
time() - start

In [ ]:
torch.cuda.empty_cache()

In [ ]:
"consider to save 'hits', which will take long"
hits_name = "hits_with_split"
if os.path.isfile(hits_name):
    print("change it")
else:
    compressed_pickle(hits_name, hits)

In [ ]:
# append prediction into the json / python dict / data frame so that it's more readable

In [ ]:
" from Sam "

In [88]:
#再寫一個幫bi_encoder算分的
falsecounter = 0
totalcounter = 0
for i in range(len(Total_Predict)):
    if Total_Predict['claim'][i].count("，")==1000:
        continue
    else:
        truebool = [0]*len(Total_Predict['evidence'][i])
        for j in range(len(Total_Predict['evidence'][i])):
            #print(i)
            if Total_Predict['label'][i]=="NOT ENOUGH INFO":
                """if Total_Predict['predicted_evidence'][i]!=[]:
                    truebool[j] = 1
                else:
                    print("fuckfuck")"""
                continue
            else:
                for k in range(len(Total_Predict['evidence'][i][j])):
                    target = [Total_Predict['evidence'][i][j][k][2],Total_Predict['evidence'][i][j][k][3]]
                    if not target in Total_Predict['predicted_evidence'][i][:5]:
                        truebool[j]=1
        if not 0 in truebool:
            falsecounter+=1
        totalcounter+=1
print((float)(1-falsecounter/totalcounter))
print(totalcounter)

0.8427921353138147
11647


In [ ]:
start = time()
#prediction_list = []
#for ans_idx in range(0, len(train), bs):
"remember, count of correct does not include not enough info yet"
"although, not enough info is already excluded from below code for now"
count_correct = 0

for ans_idx in range(0, len(validation_data)):

    prediction_list = lookup_title(hits[ans_idx], top_k=top_k, print_it=False)
    
#     # compare to the answer
#     answer = answer_of_train[ans_idx]
#     # 先: 全對才算對
#     # print(prediction_list)
#     # print(answer)
#     for ans in answer:
#         if ans in prediction_list:
#             count_correct += 1
    
torch.cuda.empty_cache()

In [ ]:
print(f"top_k : {top_k}")
print(f"correct: {count_correct}")
print(f"number of tested data so far:{ans_idx}")
print(f"correct_count/total length:{count_correct/len(answer_of_train)}")

"time spent: %.3f sec" % (time() - start)

## 感覺還行, 不過這個是沒有train test split的版本

In [80]:
top_k = 10

In [84]:
# produce question embedding
question_embedding = Bi_encoder.encode(train[0]['claim'], convert_to_tensor=True)
# optional, semantic_search will convert
#  question_embedding to the same device as corpus_embeddings
question_embedding = question_embedding.cuda() 

In [85]:
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
hits = hits[0]  # Get the hits for the first query

In [90]:
train[0]['claim']

'天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。'

In [91]:
# print the correct sentence & the score
#for h in hits:
lookup_title(hits, top_k=10)

(#1)
Original Title: 三帥

The selected passage (see if line_no is right):
6	從 《 外面的世界 》 到 《 思念是一種病 》 再到 《 牛仔很忙 》 ， 他們的配合越來越默契 ， 進步也越來越明顯 。	

(#2)
Original Title: 天衛三

The selected passage (see if line_no is right):
3	它的軌道位於天王星的內部磁層 。	天王星	天王星	磁層	磁層

(#3)
Original Title: 天衛四

The selected passage (see if line_no is right):
2	天衛四奧伯隆的名稱來自於 《 仲夏夜之夢 》 及歐洲民間傳說中的妖精之王奧伯隆 。	仲夏夜之夢	仲夏夜之夢	奧伯隆	奧伯隆

(#4)
Original Title: 盛夏獅王

The selected passage (see if line_no is right):
0	《 盛夏獅王 》  是由蒂姆 · 麥肯萊斯指導的2003年的美國電影 。	

(#5)
Original Title: 天衛六

The selected passage (see if line_no is right):
8	橢球形的天衛六軸比 （ 長軸與短軸之比 ） 爲 0.7 ± 0.2 。	

(#6)
Original Title: 鄭宜農

The selected passage (see if line_no is right):
6	同年12月 ， 她與滅火器樂團主唱楊大正結婚 ； 兩人後於2016年1月底離婚 。	

(#7)
Original Title: 天王星氣候

The selected passage (see if line_no is right):
8	事實上 ， 天王星的熱流只有 瓦 / 米 ² ， 比地球的內熱 0.075 瓦 / 米 ² 還要低 。	

(#8)
Original Title: 天衛三

The selected passage (see if line_no is right):
0	天衛三 （ 緹坦妮雅 、 Titania 、 -LSB- link - entaɪˈteɪniə -RSB- 

[['三帥', 6],
 ['天衛三', 3],
 ['天衛四', 2],
 ['盛夏獅王', 0],
 ['天衛六', 8],
 ['鄭宜農', 6],
 ['天王星氣候', 8],
 ['天衛三', 0],
 ['白種元的快餐車', 1],
 ['天衛四', 3]]

In [93]:
train[0]['evidence']

[[[4209, 4331, '天衛三', 2]]]

### predictions

In [112]:
flattened_wiki[find_le(lookup_title_idx, hits[1]['corpus_id'])]['lines'].split('\n')[0]

'0\t天衛三 （ 緹坦妮雅 、 Titania 、 -LSB- link - entaɪˈteɪniə -RSB- ） 是天王星最大的衛星 ， 也是太陽系內第八大的衛星 。\t天王星\t天王星\t太陽系\t太陽系\t第八大的衛星\t太陽系天然衛星列表\t緹坦妮雅\t緹坦妮雅'

In [113]:
flattened_wiki[find_le(lookup_title_idx, hits[1]['corpus_id'])]['lines'].split('\n')[3]

'3\t它的軌道位於天王星的內部磁層 。\t天王星\t天王星\t磁層\t磁層'

### answer

In [115]:
flattened_wiki[find_le(lookup_title_idx, hits[1]['corpus_id'])]['lines'].split('\n')[2]

'2\t1787年由威廉 · 赫歇爾發現 ， 並以威廉 · 莎士比亞的 《 仲夏夜之夢 》 中的妖精王后緹坦妮雅命名 。\t仲夏夜之夢\t仲夏夜之夢\t緹坦妮雅\t緹坦妮雅'

### check how well is this bi-encoder
### by checking the returned title and line number

In [117]:
from gc import collect
collect()

4469

In [119]:
# first, load the whole training data
train = get_jsonl_data('public_train_0316.jsonl') + get_jsonl_data('public_train_0522.jsonl')
len(train)

11620

In [126]:
train[0]['evidence'][0]

[[4209, 4331, '天衛三', 2]]

In [124]:
bi_predictions = lookup_title(hits, top_k=10, print_it=False)
bi_predictions

[['三帥', 6],
 ['天衛三', 3],
 ['天衛四', 2],
 ['盛夏獅王', 0],
 ['天衛六', 8],
 ['鄭宜農', 6],
 ['天王星氣候', 8],
 ['天衛三', 0],
 ['白種元的快餐車', 1],
 ['天衛四', 3]]

In [131]:
ele

{'id': 2663,
 'label': 'refutes',
 'claim': '天衛三軌道在天王星內部的磁層，以《 仲夏夜之夢 》作者緹坦妮雅命名。',
 'evidence': [[[4209, 4331, '天衛三', 2]]]}

In [145]:
count_None = 0
answer_of_train = []

for ele in train:
    ele = ele['evidence']
#     print(ele)
#     continue
    # ele: list of evidence
    # evidence: list of [title, line number]
    
    # but first, we have to remove the 'id's and None
    if None in ele[0]:
        count_None += 1
        continue
        
    # loop thru each evidence here
    # use sorted to compare evidence list
    evid_list = []
    for evidence in ele:
        for e in evidence:
            evid_list.append(e[2:])
    answer_of_train.append(evid_list)
    
answer_of_train[:10]

[[['天衛三', 2]],
 [['信天翁科', 2]],
 [['南大附中', 2]],
 [['毒魚豆', 4]],
 [['軟件開發', 1], ['軟件開發', 2]],
 [['國立臺灣大學應用力學研究所', 3]],
 [['威廉·倫琴', 4]],
 [['幾何學', 7]],
 [['湯姆·克魯斯', 1], ['湯姆·克魯斯', 2]],
 [['光子', 12]]]

In [148]:
answer_of_train[8] # it's shows the format we want

[['湯姆·克魯斯', 1], ['湯姆·克魯斯', 2]]

In [149]:
len(answer_of_train)

8307

In [150]:
count_None # number of None -> not enough info

3313

In [156]:
top_k

10

### 全對才算對

In [199]:
from time import time
"""
start = time()
time() - start
"""

'\nstart = time()\ntime() - start\n'

In [189]:
gpu_usage()

torch.cuda.memory_allocated: 12.770770GB
torch.cuda.memory_reserved: 14.367188GB
torch.cuda.max_memory_reserved: 29.294922GB
__CUDA Device Total Memory [GB]: 34.089730048


In [ ]:
"produce embeddings, runs fast, doesn't have to save for it"
question_embedding = Bi_encoder.encode([ele['claim'] for ele in train if None not in ele['evidence'][0]], convert_to_tensor=True)

In [297]:
top_k = 1024
top_k

1024

In [298]:
# convert embedding and hits at once since it's not that large
start = time()
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
time() - start

73.8836133480072

In [299]:
torch.cuda.empty_cache()

In [300]:
"# exclude not enough info for now"
len(hits), len(hits) == len(answer_of_train) 

(8307, True)

In [301]:
start = time()
#prediction_list = []
#for ans_idx in range(0, len(train), bs):
"remember, count of correct does not include not enough info yet"
"although, not enough info is already excluded from below code for now"
count_correct = 0

for ans_idx in range(0, len(answer_of_train)):

    # len(hits) is always = batch size
    # while, left the number of answer is not
    if ans_idx + idx > len(answer_of_train):
        torch.cuda.empty_cache()
        break
    #print("\nidx = %d" % (idx))
    prediction_list = lookup_title(hits[ans_idx], top_k=top_k, print_it=False)
    
    # compare to the answer
    answer = answer_of_train[ans_idx]
    # 先: 全對才算對
    # print(prediction_list)
    # print(answer)
    for ans in answer:
        if ans in prediction_list:
            count_correct += 1
    
torch.cuda.empty_cache()

print(f"top_k : {top_k}")
print(f"correct: {count_correct}")
print(f"number of tested data so far:{ans_idx}")
print(f"correct_count/total length:{count_correct/len(answer_of_train)}")

"time spent: %.3f sec" % (time() - start)

top_k : 1024
correct: 5985
number of tested data so far:8303
correct_count/total length:0.7204767063921993


'time spent: 24.016 sec'

### Result
#### note that we didn't count NOT_ENOUGH_INFO for now

In [302]:
ans_record = """
top_k = 10:
correct: 2100
number of tested data so far:8305
correct_count/total length:0.25279884434814015
'time spent: 0.16 sec'

top_k = 32
correct: 2920
number of tested data so far:8305
correct_count/total length:0.3515107740459853
'time spent: 0.486 sec'

top_k = 64
correct: 3540
number of tested data so far:8305
correct_count/total length:0.4261466233297219
'time spent: 0.971 sec'

top_k = 128
correct: 4123
number of tested data so far:8305
correct_count/total length:0.49632839773684845
'time spent: 1.944 sec'

top_k = 256
correct: 4762
number of tested data so far:8305
correct_count/total length:0.5732514746599253
'time spent: 4.082 sec'

top_k = 512
correct: 5345
number of tested data so far:8305
correct_count/total length:0.6434332490670519
'time spent: 8.101 sec'

top_k : 1024
correct: 5985
number of tested data so far:8303
correct_count/total length:0.7204767063921993
'time spent: 24.016 sec'""".split('\n\n')

In [291]:
def _t(idx=0):
    ttt = ans_record[idx].replace("=",":").replace(":\n","\n")
    if idx == 0:
        ttt = ttt[1:]
    ttt = ttt.split('\n')
    keys, values = [], []
    for e in ttt:
        e = e.split(":")
        keys.append(e[0])
        values.append(e[1])
    rr = [[k,v] for k,v in zip(keys, values)]
    return dict(rr)

In [293]:
_t()

{'top_k ': ' 10',
 'correct': ' 2100',
 'number of tested data so far': '8305',
 'correct_count/total length': '0.25279884434814015',
 "'time spent": " 0.16 sec'"}

In [285]:
import pandas as pd

In [303]:
rrr = []
for i in range(len(ans_record)):
    rrr.append(_t(i))
result_without_no_info = pd.DataFrame.from_dict(rrr)
result_without_no_info

,top_k,correct,number of tested data so far,correct_count/total length,'time spent
0,10,2100,8305,0.25279884434814015,0.16 sec'
1,32,2920,8305,0.3515107740459853,0.486 sec'
2,64,3540,8305,0.4261466233297219,0.971 sec'
3,128,4123,8305,0.49632839773684845,1.944 sec'
4,256,4762,8305,0.5732514746599253,4.082 sec'
5,512,5345,8305,0.6434332490670519,8.101 sec'
6,1024,5985,8303,0.7204767063921993,24.016 sec'


### 來算 標題對就算對的

In [ ]:
"not done"

In [295]:
torch.cuda.empty_cache()

In [296]:
gpu_usage()

torch.cuda.memory_allocated: 12.794491GB
torch.cuda.memory_reserved: 14.250000GB
torch.cuda.max_memory_reserved: 29.294922GB
__CUDA Device Total Memory [GB]: 34.089730048


### below needs cross encoder, skip for now

In [ ]:
# search_rewrite()
"result: title, line_no, label (supports, etc)"
def search_rewrite(query, passages=passages, top_k=top_k):
    # query: list of string
    #print_("Input question:", query)

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    # encode() will handle the case when query: str instead of list
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)

    ret = {} # dict; query: topk result
    assert len(hits) == len(query)
    for i in range(len(hits)):
      # hope there's no restriction on length of key in dict()
      ret[query[i]] = _search(hits[i], query[i], passages)
    
    return ret

In [ ]:
"we need cross encoder to run below ..."

In [ ]:
"hand-in all the result of 1 query"
def _search(hits, query, passages):
    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]
    # print_ query
    print_("\nInput question:", query)
    # Output of top-5 hits from bi-encoder
    print_("\n-------------------------\n")
    print_("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:3]:
        print_("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print_("\n-------------------------\n")
    print_("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:3]:
        print_("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))
    return hits # for title lookup

train = get_jsonl_data('public_train_0316.jsonl')
len(train)

In [ ]:
"test on 0th train data again"
pred_0 = search_rewrite([train[0]['claim']], passages, top_k=top_k)
for pred in pred_0.values():
    res = lookup_title(pred, print_it=True, wiki=flattened_wiki)
res

#### continue

In [ ]:
res = []
resans = []
for ele in train1:
  # if not related -> skip
  if ele['label'] == 'NOT ENOUGH INFO':
    """query = ele['claim']
    singleans = ele['label']
    for iii in range(2):
      #print(ele['id'])
      txt = Total_Predict['predicted_evidence'][Total_Predict['id']==ele['id']].tolist()[0]
      print(txt)
      res.append( (query, txt) )
      resans.append(label2int(singleans))"""
    continue
  # else: do the parse
  query = ele['claim']
  singleans = ele['label']
  for evidence in ele['evidence']:
    # parse list of evidence to 1 sentence
    txt = parse_evidence(evidence)
    #neg = random.choice(claimlist)
    #if neg==txt:
    #  neg = random.choice(claimlist)
    res.append( (query, txt) )
    resans.append(label2int(singleans))

fuck it
fuck it
fuck it


In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [ ]:
test_sample = []
for rrr in range(len(res)):
  test_sample.append(InputExample(texts=[res[rrr][0],res[rrr][1]], label=(float)(resans[rrr]/2)))
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_sample, batch_size=16)
#test_evaluator(model)

In [ ]:
!mkdir test_result_on_bi_encoder_500k_data
!ls

bi_encoder_500k_data			  public_submission_template.jsonl
cleaned_wiki.pbz2			  public_test_data.jsonl
cross_train_data.pbz2			  public_train_0316.jsonl
drive					  public_train_0522.jsonl
FACT_CHECK_COMP_DATA			  sample_data
prediction_results			  test_result_on_bi_encoder_500k_data
private_test_data.jsonl			  ░V╜m╕Ω«╞╢░
public_private_submission_template.jsonl  wiki-pages


In [ ]:
test_evaluator(Bi_encoder, 'test_result_on_bi_encoder_500k_data')

0.18725061361684914

In [ ]:
!ls test_result_on_bi_encoder_500k_data

similarity_evaluation_results.csv


In [ ]:
!cat test_result_on_bi_encoder_500k_data/similarity_evaluation_results.csv

epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
-1,-1,0.19259108293615715,0.18708003131939616,0.18758910102557247,0.18708044649968333,0.18746466628479208,0.18725061361684914,0.19259108006244882,0.18708090689049545


In [ ]:
import pandas as pd
pd.read_csv("test_result_on_bi_encoder_500k_data/similarity_evaluation_results.csv")

,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,-1,-1,0.192591,0.18708,0.187589,0.18708,0.187465,0.187251,0.192591,0.187081


In [ ]:
torch.cuda.empty_cache()

### Above were done

In [ ]:
test_samples = []
for ele in traintest:
  if ele['label'] == 'NOT ENOUGH INFO':
    continue
  # else: do the parse
  else:
    query = ele['claim']
    singleans = ele['label']
    for evidence in ele['evidence']:
      # parse list of evidence to 1 sentence
      txt = parse_evidence(evidence)
      test_samples.append(InputExample(texts=[query, txt], label=label2int(singleans)))
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=16)
test_evaluator(model)

fuck it


0.20982662406333022

In [ ]:
#要一個算分系統
truepredict = 0
totalpredict = 0
for i in range(len(res)):
  singleans = Cross_Encoder.predict([res[i][0],res[i][1]])
  finalans = np.argmax(singleans)
  if finalans==resans[i]:
    truepredict+=1
  totalpredict+=1
  #print(totalpredict)
print((float)(truepredict/totalpredict))

0.9404802549220439


In [ ]:
#要一個算分系統
fuckinglist = []
truepredictver2 = 0
totalpredictver2 = 0
for i in range(len(res)):
  claimsplit = res[i][0].split("，")
  singleclaimvotes = [2]*len(claimsplit)
  for j in range(len(claimsplit)):
    singleans = Cross_Encoder.predict([claimsplit[j],res[i][1]])
    singleclaimvotes[j]=np.argmax(singleans)
  #這邊可以試試不同投票模式
  fuckinglist.append(singleclaimvotes)
  if 0 in singleclaimvotes:
    finalans=0
  elif 1 in singleclaimvotes:
    finalans=1
  else:
    finalans=2
  if finalans==resans[i]:
    truepredictver2+=1
    
  totalpredictver2+=1
  #print(totalpredict)
print((float)(truepredictver2/totalpredictver2))

0.48986046511627906


In [ ]:
forcalc = np.array(resans)
print([np.sum(forcalc==0), np.sum(forcalc==1), np.sum(forcalc==2)])

[1737, 3638, 0]


In [ ]:
truefuck=0
totalfuck=0
classespredicted = [0,0,0]
for y in range(len(fuckinglist)):
  www = np.array(fuckinglist[y])
  finalans = 2*np.sum(www==1)-np.sum(www==0)+np.sum(www==2)
  if finalans>0:
    classespredicted[1]+=1
    if resans[y]==1:
      truefuck+=1
  if finalans==0:
    classespredicted[0]+=1
    if resans[y]==2:
      truefuck+=1
  if finalans<0:
    classespredicted[2]+=1
    if resans[y]==0:
      truefuck+=1
  totalfuck+=1
print((float)(truefuck/totalfuck))
print(classespredicted)

0.7118139534883721
[156, 4890, 329]


In [ ]:
truefuck=0
totalfuck=0
for y in range(len(fuckinglist)):
  www = np.array(fuckinglist[y])
  fuckvotes = [np.sum(www==0), np.sum(www==1), np.sum(www==2)]
  finalans = np.argmax(np.array(fuckvotes))
  if finalans==resans[y]:
    truefuck+=1
  totalfuck+=1
print((float)(truefuck/totalfuck))

0.4186046511627907


In [ ]:
len(Total_Predict)

11647

In [ ]:
#還要再寫一個幫bi_encoder算分的
falsecounter = 0
totalcounter = 0
for i in range(len(Total_Predict)):
  if Total_Predict['claim'][i].count("，")==1000:
    continue
  else:
    truebool = [0]*len(Total_Predict['evidence'][i])
    for j in range(len(Total_Predict['evidence'][i])):
      #print(i)
      if Total_Predict['label'][i]=="NOT ENOUGH INFO":
        """if Total_Predict['predicted_evidence'][i]!=[]:
          truebool[j] = 1
        else:
          print("fuckfuck")"""
        continue
      else:
        for k in range(len(Total_Predict['evidence'][i][j])):
          target = [Total_Predict['evidence'][i][j][k][2],Total_Predict['evidence'][i][j][k][3]]
          if not target in Total_Predict['predicted_evidence'][i][:5]:
            truebool[j]=1
    if not 0 in truebool:
      falsecounter+=1
    totalcounter+=1
print((float)(1-falsecounter/totalcounter))
print(totalcounter)

0.8427921353138147
11647


In [ ]:
len(Total_Predict)

3969

In [ ]:
for i in range(len(Total_Predict['predicted_evidence'])):
  if len(Total_Predict['predicted_evidence'][i])==0:
    print(Total_Predict.loc[i])

In [ ]:
Total_Predict.loc[35]

id                                          2888
label                            NOT ENOUGH INFO
claim                                     Claim1
evidence              [[2556, None, None, None]]
predicted_pages                               []
predicted_evidence                            []
Name: 35, dtype: object

In [ ]:
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
Total_Predict['predicted_evidence'][1]

[['靈長大目', 0],
 ['真靈長大目', 0],
 ['真靈長大目', 5],
 ['真靈長大目', 3],
 ['靈長大目', 16],
 ['真靈長大目', 4],
 ['靈長大目', 5],
 ['目_(生物)', 0],
 ['靈長大目', 3],
 ['目_(生物)', 5],
 ['靈長大目', 11],
 ['靈長大目', 13],
 ['目_(生物)', 3],
 ['靈長大目', 7],
 ['靈長大目', 9],
 ['目_(生物)', 6],
 ['目_(生物)', 4],
 ['目_(生物)', 9],
 ['目_(生物)', 11],
 ['目_(生物)', 10]]

In [ ]:
Total_Predict.loc[3941]#2051+1918 = 3969

id                                                                 7553
label                                                          supports
claim                                印度人發現毒魚豆的提取物可以令魚麻醉安靜 ， 讓他們可以徒手抓魚 。
evidence                                       [[[6845, 6563, 毒魚豆, 4]]]
predicted_pages                                           [印度人, 毒魚豆, 魚]
predicted_evidence    [[毒魚豆, 4], [毒魚豆, 0], [毒魚豆, 6], [毒魚豆, 5], [毒魚豆,...
Name: 3941, dtype: object

# Note
1. 其實原本的baseline code 抓evidence抓得滿準的，甚至不需要apply cross encoder(原本的前五名就有62%的準度了，輸出32條可以拉到65%)

2. Baseline Code可以調他的輸出量(預設是輸出五條，但我改成32之後準度就上升了，從58%->62%)

3. 我們自己的bi_encoder準確度只有8%(輸出32條的話可以到12%)

4. 現在的cross_encoder(for NLI)準度有91%

5. 應該先把Claim拆解再去讀問題

In [ ]:
#Cross_Encoder.save("NLI_Val_67")
bi_encoder.save("bi_mpnet_e1")

In [ ]:
!zip -r /content/bi_mpnet_e1_Model.zip /content/bi_mpnet_e1

  adding: content/bi_mpnet_e1/ (stored 0%)
  adding: content/bi_mpnet_e1/vocab.txt (deflated 53%)
  adding: content/bi_mpnet_e1/2_Normalize/ (stored 0%)
  adding: content/bi_mpnet_e1/README.md (deflated 53%)
  adding: content/bi_mpnet_e1/special_tokens_map.json (deflated 50%)
  adding: content/bi_mpnet_e1/config.json (deflated 48%)
  adding: content/bi_mpnet_e1/sentence_bert_config.json (deflated 4%)
  adding: content/bi_mpnet_e1/config_sentence_transformers.json (deflated 27%)
  adding: content/bi_mpnet_e1/pytorch_model.bin (deflated 8%)
  adding: content/bi_mpnet_e1/tokenizer_config.json (deflated 49%)
  adding: content/bi_mpnet_e1/1_Pooling/ (stored 0%)
  adding: content/bi_mpnet_e1/1_Pooling/config.json (deflated 47%)
  adding: content/bi_mpnet_e1/tokenizer.json (deflated 71%)
  adding: content/bi_mpnet_e1/modules.json (deflated 62%)


In [ ]:
from google.colab import files
files.download("Hahaha_Model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!gdown --folder https://drive.google.com/drive/u/1/folders/1-HQC9UzEYcyM5VbSxBJqONawmVW6D20l?fbclid=IwAR38oIhEC5UzFet_d02YNYCHGdd26Ixl4yNDFV1vnymae1og7IppLBN_GA8

In [ ]:
with open('prediction_results/test_doc64sent64_with_private.jsonl',encoding='utf8') as test_source:
  Test_df = pd.read_json(test_source, lines=True)

In [ ]:
Test_df

In [ ]:
with open("public_private_submission_template.jsonl", encoding = 'utf8') as thetemplate:
  subtemplate = pd.read_json(thetemplate, lines = True)

In [ ]:
subtemplate

,id,predicted_label,predicted_evidence
0,63,not enough info,NaN
1,74,not enough info,NaN
2,77,not enough info,NaN
3,78,not enough info,NaN
4,82,not enough info,NaN
...,...,...,...
9033,22111,not enough info,NaN
9034,22112,not enough info,NaN
9035,22113,not enough info,NaN
9036,22114,not enough info,NaN


In [ ]:
Test_df['predicted_evidence'][Test_df['id']==subtemplate['id'][100]][:5].tolist()[0]

In [ ]:
subtemplate['predicted_evidence'][100]

[['馬丁·路德·金', 5],
 ['馬丁·路德·金', 0],
 ['馬丁·路德·金國家歷史遺址', 1],
 ['伯納德·蒙哥馬利', 1],
 ['馬丁·路德·金紀念日', 5]]

In [ ]:
final_evidence = []
for i in range(len(subtemplate['id'])):
  final_evidence.append(Test_df['predicted_evidence'][Test_df['id']==subtemplate['id'][i]].tolist()[0][:5])
final_evidence

In [ ]:
subtemplate['predicted_evidence'] = final_evidence

In [ ]:
subtemplate

,id,predicted_label,predicted_evidence
0,63,not enough info,"[[女真, 4], [女真, 1], [人, 5], [人, 7], [漢姓, 0]]"
1,74,not enough info,"[[昭和天皇, 16], [昭和天皇, 0], [昭和, 0], [昭和天皇, 4], [天..."
2,77,not enough info,"[[昭和天皇, 0], [昭和天皇, 16], [昭和天皇, 4], [昭和, 0], [天..."
3,78,not enough info,"[[昭和天皇, 16], [昭和天皇, 0], [1989, 5], [昭和天皇, 4], ..."
4,82,not enough info,"[[唐朝, 4], [大象_(年號), 0], [象, 0], [象, 1], [唐朝, 3]]"
...,...,...,...
9033,22111,not enough info,"[[張惠妹, 0], [張惠妹, 3], [張惠妹, 9], [張惠妹, 8], [張惠妹,..."
9034,22112,not enough info,"[[張惠妹, 13], [張惠妹, 0], [張惠妹, 14], [張惠妹, 7], [張惠..."
9035,22113,not enough info,"[[張惠妹, 13], [張惠妹, 0], [張惠妹, 14], [張惠妹, 7], [張惠..."
9036,22114,not enough info,"[[張惠妹, 13], [張惠妹, 0], [張惠妹, 14], [張惠妹, 7], [張惠..."


In [ ]:
finallabels = []
fuckidx = 0
for i in range(len(Test_df)):
  #votes = [0,0,0]
  currentsentence=""
  for singleevidence in subtemplate['predicted_evidence'][i]:
    #print(singleevidence)
    original = flattened_wiki[title_to_idx[singleevidence[0]]]['lines'].split('\n')[singleevidence[1]]
    start_idx = original.index('\t')
    if "。" in original:
      end_idx = original.index("。")
    else:
      end_idx = len(original)
    sentence = original[start_idx:end_idx].replace('\t','').replace(' ','')
    currentsentence+=sentence
  originalans = Cross_Encoder.predict([Test_df['claim'][Test_df['id']==subtemplate['id'][i]].tolist()[0],currentsentence])
  finalans = np.argmax(originalans)
  if finalans==0:
    finallabels.append("refutes")
  if finalans==1:
    finallabels.append("NOT ENOUGH INFO")
  if finalans==2:
    finallabels.append("supports")

In [ ]:
subtemplate['predicted_label'] = finallabels

In [ ]:
for i in range(len(subtemplate['predicted_label'])):
  if subtemplate['predicted_label'][i]=="NOT ENOUGH INFO":
    subtemplate['predicted_evidence'][i] = "None"

<ipython-input-58-c41fe23c2646>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subtemplate['predicted_evidence'][i] = "None"


In [ ]:
for i in range(len(subtemplate['predicted_label'])):
  if subtemplate['predicted_evidence'][i]==[]:
    print("幹")
    subtemplate['predicted_label'][i] = "NOT ENOUGH INFO"

幹


<ipython-input-59-7bb16938b527>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subtemplate['predicted_label'][i] = "NOT ENOUGH INFO"


In [ ]:
subtemplate.to_json("submission_500060_64_combine_5concat.jsonl", orient="records",lines=True,force_ascii=False)

In [ ]:
from google.colab import files
files.download("submission_500060_64_combine_5concat.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
forcalc2 = np.array(finallabels)

In [ ]:
np.sum(forcalc2=="refutes")

377

In [ ]:
np.sum(forcalc2=="supports")

526

In [ ]:
np.sum(forcalc2=="NOT ENOUGH INFO")

86

In [ ]:
Cross_Encoder.predict(["地球是行星，月球是狗", "我家的狗叫地球,月球也是一隻狗，天上有兩顆太陽。這個太陽系裡的成員有九個，天氣非常良好，大谷翔平今天投了一場無安打比賽，但天使隊的成績還是像狗屎一樣"])

array([-3.8613591,  4.7201514, -2.2126446], dtype=float32)

In [ ]:
Cross_Encoder.predict(["地球是行星", "我家有養一隻狗"], apply_softmax=True)

array([2.059870e-03, 9.978684e-01, 7.167068e-05], dtype=float32)

In [ ]:
#只要有refutes，他就該是refutes
#只有supprots不代表是supprots！
#注意：一句話有很多部分，他只要是partial supports就會回傳supports給你，partial refuse也會回傳refutes給你，你不知道到底是哪一part讓他覺得supprots，哪一part讓他覺得refutes
#拿全部資料來train的bi_encoder: 87%, 看前五=84%，其實準確度滿高！

In [ ]:
Test_Encoder = CrossEncoder("cross-encoder/qnli-distilroberta-base")

# 感覺現在Data夠了，可以試試看bi_encoder

In [ ]:
bi_encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
train_loss = losses.MultipleNegativesRankingLoss(model = bi_encoder)
bi_loader = datasets.NoDuplicatesDataLoader(cross_train_data, batch_size=16)

In [ ]:
bi_encoder.fit??

In [ ]:
bi_encoder.fit(train_objectives=[(bi_loader, train_loss)], epochs = 1, use_amp = True)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/31253 [00:00<?, ?it/s]

# 在想要不要做一個reranker
其實應該要，我們128的data可以做到85%，但只看前五筆的話就只有81%，真的要拚arrucary的話這4%應該要救

那就要再挑現成的去fine-tune，只是我不知道效果會怎麼樣就是了

而且我拿train到97的cross_encoder去predict，最終準確率居然只跟原本的一樣(51%)，我感覺是predicted evidence限制了這個model的上限！

array([0.53860414], dtype=float32)